# Segmenting and Clustering Neighbourhoods in Toronto

## Import required libraries

In [141]:
# general imports
import pandas as pd
import numpy as np

# required for pulling down web data
import urllib

# required for parsing an HTML document
from bs4 import BeautifulSoup

## Pull down a copy of the data

In [142]:
# obtain a copy of the web page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

## Parse the downloaded data

In [143]:
# parse the downloaded HTML page
soup = BeautifulSoup(page, 'html.parser')

#get a reference to the correct table
table=soup.find('table', class_='wikitable sortable')

## Extract the column names

In [144]:
#obtain the column names, stripping off whitespace
columnNames = []
for name in table.findAll( 'th' ):
    columnNames.append( name.find(text=True ).strip() )

#lets see what we've got    
print( columnNames )

['Postcode', 'Borough', 'Neighbourhood']


## Extract the data into a DataFrame

In [145]:
#location to store the data during the extraction loop
cellData = []
rowData = []

#pull out each row of data
for row in table.findAll( 'tr' ):

    #pull out each cell from the row
    for cell in row.findAll( 'td' ):
        cellData.append( cell.find(text=True).strip() )

    #only append if we hold complete and correct data (exclude if borough is 'Not assigned')
    if( len( cellData ) == len( columnNames ) ):
        if( cellData[1] != "Not assigned" ):
            rowData.append( cellData )
    cellData = []


#once we have all the cells, create a data frame with this data
df = pd.DataFrame( np.array( rowData ), columns=columnNames )

## Combine Neighbourhoods

In [146]:
#group the data by post code, combining Neighbourhood 
grouped = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join ).reset_index()

# Ensure we print out the same order as the given screenshot - not required, just for fun
grouped.loc[ (grouped.Postcode == 'M5G') | (grouped.Postcode == 'M2H') | (grouped.Postcode == 'M4B')| (grouped.Postcode == 'M1J')| (grouped.Postcode == 'M4G')| (grouped.Postcode == 'M4M')| (grouped.Postcode == 'M1R')| (grouped.Postcode == 'M9V')| (grouped.Postcode == 'M9L')| (grouped.Postcode == 'M5V')| (grouped.Postcode == 'M1B')| (grouped.Postcode == 'M5A')]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
5,M1J,Scarborough,Scarborough Village
11,M1R,Scarborough,"Maryvale,Wexford"
17,M2H,North York,Hillcrest Village
35,M4B,East York,"Woodbine Gardens,Parkview Hill"
38,M4G,East York,Leaside
43,M4M,East Toronto,Studio District
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"
57,M5G,Downtown Toronto,Central Bay Street
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


## Replace 'Not Assigned' in Neighbourhoods

In [147]:
grouped.shape

(103, 3)

In [148]:
#validate we have a 'Not assigned' neighbourhood
print( grouped.loc[grouped.Neighbourhood == 'Not assigned'] )
print()

#when the neighbourhood is 'Not assigned', replace with the borough
grouped.Neighbourhood = pd.np.where(grouped.Neighbourhood == 'Not assigned', grouped.Borough, grouped.Neighbourhood)

#validate we do NOT have a 'Not assigned' neighbourhood
print( grouped.loc[grouped.Neighbourhood == 'Not assigned'])
print()

   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []



# START OF NOTEBOOK 2

---

## Combine geospatial data with the existing DataFrame

In [149]:
geospatial = {
'Postcode' : [ 'M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N', 'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H', 'M2J', 'M2K', 'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B', 'M3C', 'M3H', 'M3J', 'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B', 'M4C', 'M4E', 'M4G', 'M4H', 'M4J', 'M4K', 'M4L', 'M4M', 'M4N', 'M4P', 'M4R', 'M4S', 'M4T', 'M4V', 'M4W', 'M4X', 'M4Y', 'M5A', 'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J', 'M5K', 'M5L', 'M5M', 'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W', 'M5X', 'M6A', 'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L', 'M6M', 'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W', 'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P', 'M9R', 'M9V', 'M9W' ],
'Latitude' : [ 43.8066863, 43.7845351, 43.7635726, 43.7709921, 43.773136, 43.7447342, 43.7279292, 43.7111117, 43.716316, 43.692657, 43.7574096, 43.7500715, 43.7942003, 43.7816375, 43.8152522, 43.7995252, 43.8361247, 43.8037622, 43.7785175, 43.7869473, 43.7574902, 43.789053, 43.7701199, 43.7527583, 43.7827364, 43.7532586, 43.7459058, 43.7258997, 43.7543283, 43.7679803, 43.7374732, 43.7390146, 43.7284964, 43.7616313, 43.7258823, 43.7063972, 43.6953439, 43.6763574, 43.7090604, 43.7053689, 43.685347, 43.6795571, 43.6689985, 43.6595255, 43.7280205, 43.7127511, 43.7153834, 43.7043244, 43.6895743, 43.6864123, 43.6795626, 43.667967, 43.6658599, 43.6542599, 43.6571618, 43.6514939, 43.6447708, 43.6579524, 43.6505712, 43.6408157, 43.6471768, 43.6481985, 43.7332825, 43.7116948, 43.6969476, 43.6727097, 43.6626956, 43.6532057, 43.6289467, 43.6464352, 43.6484292, 43.718518, 43.709577, 43.6937813, 43.6890256, 43.669542, 43.6690051, 43.6479267, 43.6368472, 43.7137562, 43.6911158, 43.6731853, 43.6616083, 43.6489597, 43.6515706, 43.6623015, 43.6369656, 43.6627439, 43.6056466, 43.6024137, 43.6536536, 43.6362579, 43.6288408, 43.6678556, 43.6509432, 43.6435152, 43.7563033, 43.7247659, 43.706876, 43.696319, 43.6889054, 43.7394164, 43.7067483 ],
'Longitude' : [ -79.1943534, -79.1604971, -79.1887115, -79.2169174, -79.2394761, -79.2394761, -79.2620294, -79.2845772, -79.2394761, -79.2648481, -79.273304, -79.2958491, -79.2620294, -79.3043021, -79.2845772, -79.3183887, -79.2056361, -79.3634517, -79.3465557, -79.385975, -79.3747141, -79.4084928, -79.4084928, -79.4000493, -79.4422593, -79.3296565, -79.352188, -79.340923, -79.4422593, -79.4872619, -79.4647633, -79.5069436, -79.4956974, -79.5209994, -79.3155716, -79.309937, -79.3183887, -79.2930312, -79.3634517, -79.3493719, -79.3381065, -79.352188, -79.3155716, -79.340923, -79.3887901, -79.3901975, -79.4056784, -79.3887901, -79.3831599, -79.4000493, -79.3775294, -79.3676753, -79.3831599, -79.3606359, -79.3789371, -79.3754179, -79.3733064, -79.3873826, -79.3845675, -79.3817523, -79.3815764, -79.3798169, -79.4197497, -79.4169356, -79.4113072, -79.4056784, -79.4000493, -79.4000493, -79.3944199, -79.374846, -79.3822802, -79.4647633, -79.4450726, -79.4281914, -79.453512, -79.4225637, -79.4422593, -79.4197497, -79.4281914, -79.4900738, -79.4760133, -79.4872619, -79.4647633, -79.456325, -79.4844499, -79.3894938, -79.615819, -79.321558, -79.5013207, -79.5434841, -79.5069436, -79.4985091, -79.5209994, -79.5322424, -79.5547244, -79.5772008, -79.5659633, -79.5322424, -79.5181884, -79.5322424, -79.5547244, -79.5884369, -79.5940544 ]
}

geospatial_df = pd.DataFrame( geospatial, columns=['Postcode', 'Latitude', 'Longitude'] )

#join this geo data with the existing dataframe
joined_df = pd.merge( grouped, geospatial_df, on='Postcode', how='outer')

# Ensure we print out the same order as the given screenshot - not required, just for fun
joined_df.loc[ (joined_df.Postcode == 'M5G') | (joined_df.Postcode == 'M2H') | (joined_df.Postcode == 'M4B')| (joined_df.Postcode == 'M1J')| (joined_df.Postcode == 'M4G')| (joined_df.Postcode == 'M4M')| (joined_df.Postcode == 'M1R')| (joined_df.Postcode == 'M9V')| (joined_df.Postcode == 'M9L')| (joined_df.Postcode == 'M5V')| (joined_df.Postcode == 'M1B')| (joined_df.Postcode == 'M5A')]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
11,M1R,Scarborough,"Maryvale,Wexford",43.750071,-79.295849
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
35,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
38,M4G,East York,Leaside,43.709060,-79.363452
43,M4M,East Toronto,Studio District,43.659526,-79.340923
53,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420


In [150]:
# install folium for maps
!conda install -c conda-forge folium=0.5.0 --yes

import folium
from folium.plugins import MarkerCluster

import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Fancy colours!
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

# START OF NOTEBOOK 3

---

## Explore and cluster the neighborhoods in Toronto

## Extract out the lat longs so we can eyeball them

In [151]:
locations = joined_df[ ['Latitude', 'Longitude'] ]
locationlist = locations.values.tolist()

# build a map, starting at one of the given lat longs
map = folium.Map(location=[43.8066863, -79.1943534], zoom_start=11)

# marker clustering
marker_cluster = folium.plugins.MarkerCluster().add_to(map)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=joined_df['Postcode'][point]).add_to(marker_cluster)

# draw the map to see what we have
map

## Foursquare config

In [152]:
# Use foursquare to find out some interesting facts
CLIENT_ID = 'DKFWKQVXCQTKUWZPXBQ3IY55QNQ42ZDL1UCPATS11UHYKLHU' # your Foursquare ID
CLIENT_SECRET = '4WBPTEX5O4AMV3CS4DYQGGHYF35B5VS2BXGONVAHX1RKM5YH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Define a function to locate venues - duly stolen from the hands on lab

In [153]:
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            10)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Call this function to get venue information

In [154]:
venues = getNearbyVenues(names=joined_df['Postcode'], latitudes=joined_df['Latitude'], longitudes=joined_df['Longitude'] )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


## One hot encoding

In [155]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Postcode'] = venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

grouped = onehot.groupby('Postcode').mean().reset_index()
grouped

,Postcode,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bookstore,Breakfast Spot,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,M1J,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
1,M1W,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
2,M2J,0.0,0.000000,0.0,0.333333,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.666667,0.0,0.000,0.0,0.0,0.000000
3,M3B,0.0,0.000000,0.0,0.000000,0.000000,0.5,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
4,M3C,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
5,M3H,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
6,M4E,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,1.0,0.0,0.000000
7,M4G,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
8,M4H,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000
9,M4J,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.000000


## Create a sorted dataset of top venues

In [156]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Postcode'] = grouped['Postcode']

for ind in np.arange(grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1J,Playground,Yoga Studio,Hot Dog Joint,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Food Court
1,M1W,Fast Food Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Chinese Restaurant,Pharmacy,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market
2,M2J,Theater,Bank,Yoga Studio,Food Court,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant
3,M3B,Pool,Baseball Field,Yoga Studio,Food Court,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Fried Chicken Joint
4,M3C,Fast Food Restaurant,Yoga Studio,Vegetarian / Vegan Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Food Court,Fried Chicken Joint


## Setup clusters and merge the datasets

In [157]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = pd.merge( joined_df, venues_sorted, on='Postcode', how='inner')

merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Yoga Studio,Hot Dog Joint,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Food Court
1,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,4,Fast Food Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Chinese Restaurant,Pharmacy,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market
2,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,4,Theater,Bank,Yoga Studio,Food Court,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,4,Pool,Baseball Field,Yoga Studio,Food Court,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Fried Chicken Joint
4,M3C,North York,"Flemingdon Park,Don Mills South",43.725900,-79.340923,4,Fast Food Restaurant,Yoga Studio,Vegetarian / Vegan Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Farmers Market,Food Court,Fried Chicken Joint


## Finally, render what we have

In [158]:
# create map
map_clusters = folium.Map(location=[43.744734, -79.239476], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Postcode'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters